In [21]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

# As there are possibilities of different chrome
# browser and we are not sure under which it get
# executed let us use the below syntax
from selenium.webdriver.chrome.options import Options

def google_review_parser():
    options = Options()
    options.add_argument('--headless')

    serv = webdriver.chrome.service.Service('/Users/sgo/Doc/Jupyter/cxNLP/chromedriver')

    driver = webdriver.Chrome(service=serv, options=options)

    # search
    search = '吾米食肆'

    search = '+'.join(search.split(' '))

    url = f'https://www.google.com/maps/place/{search}/?hl=en'
    driver.get(url)

    # process of parsing reviews
    driver.find_element(by=By.XPATH,
                        value='//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()

    # button
    driver.find_element(by=By.XPATH, 
                        value='//*[@id="searchbox-searchbutton"]').click()
    time.sleep(2)

    # button
    driver.find_element(by=By.CLASS_NAME, 
                        value='DkEaL').click()
    time.sleep(2)

    # get number of reivews
    total_number_of_reviews = driver.find_element(By.XPATH, "//*[@class='fontBodySmall']").text.split(" ")[0]
    total_number_of_reviews = int(total_number_of_reviews.replace('.','')) if '.' in total_number_of_reviews else int(total_number_of_reviews)

    #Find scroll layout
    scrollable_div = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")

    count = 0
    #Scroll as many times as necessary to load all reviews
    for i in range(0,(round(total_number_of_reviews/10 - 1))):
        more = driver.find_elements(by=By.XPATH, 
                        value='//*[@aria-label=" See more "]')
        for m in more:
            m.click()
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                scrollable_div)
        time.sleep(2)
        count+=1
        if 2*count > 300: break


    # extract reviews by BeatifulSoup
    response = BeautifulSoup(driver.page_source, 'html.parser')
    texts = response.find_all('span', class_='wiI7pd')
    rates = response.find_all('div', class_='DU9Pgb')


    def get_review_summary(text_set, rate_set):
        rev_dict = {'Review Text': [],
                   'Review Rate': [],
                   'Review Time': []}
        for text, rate in zip(text_set, rates):
            review_rate = rate.find('span', class_='kvMYJc')["aria-label"].strip()[0]
            review_time = rate.find('span',class_='rsqaWe').text
            review_text = text.text
            rev_dict['Review Rate'].append(review_rate)
            rev_dict['Review Time'].append(review_time)
            rev_dict['Review Text'].append(review_text)
        import pandas as pd    
        return(pd.DataFrame(rev_dict))

    df = get_review_summary(texts, rates)

    from datetime import date

    df['search']=search
    df['insert_date']=date.today()
    
    df.to_csv(f'{search}.csv', index=False)
    return df

google_review_parser()

,Review Text,Review Rate,Review Time,search,insert_date
0,(Translated by Google) Ginger Ginger Roast Por...,5,4 months ago,吾米食肆,2022-10-03
1,"(Translated by Google) Ubereats delivery, 45 m...",5,6 months ago,吾米食肆,2022-10-03
2,(Translated by Google) After walking through t...,5,9 months ago,吾米食肆,2022-10-03
3,(Translated by Google) Ubereats delivery\nI or...,5,11 months ago,吾米食肆,2022-10-03
4,(Translated by Google) Very clean and deliciou...,5,a year ago,吾米食肆,2022-10-03
...,...,...,...,...,...
105,,5,2 years ago,吾米食肆,2022-10-03
106,,5,a year ago,吾米食肆,2022-10-03
107,,4,a year ago,吾米食肆,2022-10-03
108,,4,9 months ago,吾米食肆,2022-10-03


In [ ]:
# separte process

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

# As there are possibilities of different chrome
# browser and we are not sure under which it get
# executed let us use the below syntax
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
# options.add_argument('--disable-translate')
# options.add_argument("--lang=en-GB")
options.add_argument("--lang=zh-TW")



# prefs = {
#   "translate_whitelists": {"ch":"en"},
#   "translate":{"enabled":"true"}
# }
# options.add_experimental_option("prefs", prefs)

# Serv = Service('/Users/sgo/Doc/Jupyter/cxNLP/chromedriver')

serv = webdriver.chrome.service.Service('/Users/sgo/Doc/Jupyter/cxNLP/chromedriver')

driver = webdriver.Chrome(service=serv, options=options)

# driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [2]:
# from selenium import webdriver
# driver = webdriver.Chrome()
#London Victoria & Albert Museum URL
# url = 'https://www.google.com/maps/place/Victoria+and+Albert+Museum/@51.4966392,-0.17218,15z/data=!4m5!3m4!1s0x0:0x9eb7094dfdcd651f!8m2!3d51.4966392!4d-0.17218'

# search = 'Pinakothek der Moderne'
# search = 'SEEN RESTAURANT'
# search = 'Ya The Mandarin Room'
# search = "優勢皮膚科診所" #南港杏安診所 #'Dr, Yu's Skin Clinic' #'隆美皮膚科診所'
# search = 'Gushu Hotpot'
# search = 'Crispy & More'
search = '吾米食肆'

search = '+'.join(search.split(' '))

url = f'https://www.google.com/maps/place/{search}/?hl=en'
driver.get(url)

In [3]:
driver.find_element(by=By.XPATH,
                    value='//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()

# time.sleep(1)
# driver.find_element(by=By.XPATH,
#                     value='//*[@id="settings"]/div/div[2]/ul/div[7]/li[1]/button').click()

# time.sleep(1)
# driver.find_element(by=By.XPATH,
#                     value='//*[@id="languages"]/div/div[2]/div[2]/ul[1]/li[11]/a').click()

# time.sleep(1)

In [4]:
driver.find_element(by=By.XPATH, 
                    value='//*[@id="searchbox-searchbutton"]').click()
time.sleep(2)

In [5]:
driver.find_element(by=By.CLASS_NAME, 
                    value='DkEaL').click()
time.sleep(2)

In [6]:
# try:
#     driver.find_element(By.CLASS_NAME, "widget-pane-link").click()
# except Exception:
#     response = BeautifulSoup(driver.page_source, 'html.parser')
#     # Check if there are any paid ads and avoid them
#     if response.find_all('span', {'class': 'ARktye-badge'}):
#         ad_count = len(response.find_all('span', {'class': 'ARktye-badge'}))
#         li = driver.find_elements(By.CLASS_NAME, "a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd")
#         li[ad_count].click()
#     else:
#         driver.find_element(By.CLASS_NAME, "a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd").click()
#         time.sleep(5)
#     driver.find_element(By.CLASS_NAME, "widget-pane-link").click()

In [7]:
total_number_of_reviews = driver.find_element(By.XPATH, "//*[@class='fontBodySmall']").text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace('.','')) if '.' in total_number_of_reviews else int(total_number_of_reviews)

total_number_of_reviews

114

In [8]:
#Find scroll layout
scrollable_div = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")

In [9]:
count = 0
#Scroll as many times as necessary to load all reviews
for i in range(0,(round(total_number_of_reviews/10 - 1))):
    more = driver.find_elements(by=By.XPATH, 
                    value='//*[@aria-label=" See more "]')
    for m in more:
        m.click()
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
            scrollable_div)
    time.sleep(2)
    count+=1
    if 2*count > 300: break

In [10]:
response = BeautifulSoup(driver.page_source, 'html.parser')
texts = response.find_all('span', class_='wiI7pd')
rates = response.find_all('div', class_='DU9Pgb')

In [11]:
texts

[<span class="wiI7pd" jsan="7.wiI7pd" jstcache="1252">(Translated by Google) Ginger Ginger Roast Pork Uber eats$120-24=$96
 🌟🌟🌟🌟🌟
 The rice is unexpectedly delicious👍 it's close to the level of Chishang rice buns
 Rich side dishes, fresh kelp, sweet roasted meat😋
 The mushrooms in the side dishes are a bit sour, but it may be the king oyster mushroom 🍄 itself is prone to sour taste
 
 Green Curry Chicken Uber eats$110
 🌟🌟🌟🌟🌟
 The tender chicken is not chai, the curry is not greasy, the spicy is delicious, and it is missed 🥰
 Rich side dishes, just right curry, will not eat rice left behind, nor will there be a lot of sauce left at the end to deal with
 
 (Original)
 薑薑好燒肉Uber eats$120-24=$96
 🌟🌟🌟🌟🌟
 白飯意外地好吃👍有接近池上飯包的水準
 配菜豐富、海帶新鮮、燒肉香甜😋
 配菜中菇類有點酸，但可能是杏鮑菇🍄本身容易呈現酸味
 
 綠咖喱嫩雞 Uber eats$110
 🌟🌟🌟🌟🌟
 嫩雞不柴、咖哩不膩、辣得好吃、令人想念🥰
 豐富的配菜，剛剛好的咖哩，不會吃到後面剩下白飯，也不會吃到最後剩很多醬汁需要處理</span>,
 <span class="wiI7pd" jsan="7.wiI7pd" jstcache="1252">(Translated by Google) Ubereats delivery, 45 minutes
 Not much to say, t

In [12]:
texts[0].text

"(Translated by Google) Ginger Ginger Roast Pork Uber eats$120-24=$96\n🌟🌟🌟🌟🌟\nThe rice is unexpectedly delicious👍 it's close to the level of Chishang rice buns\nRich side dishes, fresh kelp, sweet roasted meat😋\nThe mushrooms in the side dishes are a bit sour, but it may be the king oyster mushroom 🍄 itself is prone to sour taste\n\nGreen Curry Chicken Uber eats$110\n🌟🌟🌟🌟🌟\nThe tender chicken is not chai, the curry is not greasy, the spicy is delicious, and it is missed 🥰\nRich side dishes, just right curry, will not eat rice left behind, nor will there be a lot of sauce left at the end to deal with\n\n(Original)\n薑薑好燒肉Uber eats$120-24=$96\n🌟🌟🌟🌟🌟\n白飯意外地好吃👍有接近池上飯包的水準\n配菜豐富、海帶新鮮、燒肉香甜😋\n配菜中菇類有點酸，但可能是杏鮑菇🍄本身容易呈現酸味\n\n綠咖喱嫩雞 Uber eats$110\n🌟🌟🌟🌟🌟\n嫩雞不柴、咖哩不膩、辣得好吃、令人想念🥰\n豐富的配菜，剛剛好的咖哩，不會吃到後面剩下白飯，也不會吃到最後剩很多醬汁需要處理"

In [13]:
def get_review_summary(text_set, rate_set):
    rev_dict = {'Review Text': [],
               'Review Rate': [],
               'Review Time': []}
    for text, rate in zip(text_set, rates):
        review_rate = rate.find('span', class_='kvMYJc')["aria-label"].strip()[0]
        review_time = rate.find('span',class_='rsqaWe').text
        review_text = text.text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)
    import pandas as pd    
    return(pd.DataFrame(rev_dict))

df = get_review_summary(texts, rates)

In [14]:
from datetime import date

df['search']=search
df['insert_date']=date.today()

df

,Review Text,Review Rate,Review Time,search,insert_date
0,(Translated by Google) Ginger Ginger Roast Por...,5,4 months ago,吾米食肆,2022-10-01
1,"(Translated by Google) Ubereats delivery, 45 m...",5,6 months ago,吾米食肆,2022-10-01
2,(Translated by Google) After walking through t...,5,9 months ago,吾米食肆,2022-10-01
3,(Translated by Google) Ubereats delivery\nI or...,5,10 months ago,吾米食肆,2022-10-01
4,(Translated by Google) Very clean and deliciou...,5,a year ago,吾米食肆,2022-10-01
...,...,...,...,...,...
105,,5,2 years ago,吾米食肆,2022-10-01
106,,5,a year ago,吾米食肆,2022-10-01
107,,4,a year ago,吾米食肆,2022-10-01
108,,4,9 months ago,吾米食肆,2022-10-01


In [15]:
df['Review Text'][0]

"(Translated by Google) Ginger Ginger Roast Pork Uber eats$120-24=$96\n🌟🌟🌟🌟🌟\nThe rice is unexpectedly delicious👍 it's close to the level of Chishang rice buns\nRich side dishes, fresh kelp, sweet roasted meat😋\nThe mushrooms in the side dishes are a bit sour, but it may be the king oyster mushroom 🍄 itself is prone to sour taste\n\nGreen Curry Chicken Uber eats$110\n🌟🌟🌟🌟🌟\nThe tender chicken is not chai, the curry is not greasy, the spicy is delicious, and it is missed 🥰\nRich side dishes, just right curry, will not eat rice left behind, nor will there be a lot of sauce left at the end to deal with\n\n(Original)\n薑薑好燒肉Uber eats$120-24=$96\n🌟🌟🌟🌟🌟\n白飯意外地好吃👍有接近池上飯包的水準\n配菜豐富、海帶新鮮、燒肉香甜😋\n配菜中菇類有點酸，但可能是杏鮑菇🍄本身容易呈現酸味\n\n綠咖喱嫩雞 Uber eats$110\n🌟🌟🌟🌟🌟\n嫩雞不柴、咖哩不膩、辣得好吃、令人想念🥰\n豐富的配菜，剛剛好的咖哩，不會吃到後面剩下白飯，也不會吃到最後剩很多醬汁需要處理"

In [16]:
today = date.today()
print("Today's date:", today)

Today's date: 2022-10-01


In [17]:
df.to_csv(f'{search}.csv', index=False)

In [18]:
f'{search}.csv'
driver.close()

In [19]:
# # ----- Example Python program to create a database in PostgreSQL using Psycopg2 -----

# # import the PostgreSQL client for Python

# import psycopg2

# from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

# # Connect to PostgreSQL DBMS
# con = psycopg2.connect(dbname='reviews', user='postgres', password='Wsxc5415', host='localhost', port='5432');
# con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

# # Obtain a DB Cursor
# cursor          = con.cursor();

# with open(f'{search}.csv', 'r') as f:
#     next(f)
# #     cursor.copy_from(f, 'google_reviews', sep=',')
#     copy = "COPY google_reviews(review, rating, date, search, insert_date) FROM STDIN with csv"
#     cursor.copy_expert(sql=copy, file=f)
#     con.commit()
